In [ ]:
import pandas as pd

# CSV 파일 로드
df = pd.read_csv('seoul_air_quality_data.csv')

# 각 열의 평균값 계산 (0인 값을 제외하고 평균을 구합니다)
pollutionData = ['NO2', 'O3', 'CO', 'SO2', 'PM10', 'PM25']
mean_values = df[pollutionData].apply(lambda x: x[x != 0].mean())

# 0값을 각 열의 평균값으로 대체
df_filled = df.copy()  # 복사본 생성
for col in pollutionData:
    df_filled[col] = df_filled[col].replace(0, mean_values[col])

# 'MSRDT' 열을 날짜/시간 형식으로 변환
df_filled['MSRDT'] = pd.to_datetime(df_filled['MSRDT'], format='%Y%m%d%H%M')

# 날짜 관련 열 추가
df_filled['year'] = df_filled['MSRDT'].dt.year
df_filled['month'] = df_filled['MSRDT'].dt.month
df_filled['day'] = df_filled['MSRDT'].dt.day
df_filled['hour'] = df_filled['MSRDT'].dt.hour
df_filled['weekday'] = df_filled['MSRDT'].dt.weekday  # 월요일=0, 일요일=6

# 좌표 데이터
coordinates = {
    "종로구": (126.981641666666, 37.5703777777777),
    "중구": (126.999641666666, 37.5610027777777),
    "용산구": (126.967522222222, 37.5360944444444),
    "성동구": (127.039, 37.5606111111111),
    "광진구": (127.084533333333, 37.5357388888888),
    "동대문구": (127.042141666666, 37.571625),
    "중랑구": (127.094777777777, 37.6038055555555),
    "성북구": (127.020333333333, 37.5863833333333),
    "강북구": (127.027719444444, 37.6369555555555),
    "도봉구": (127.049522222222, 37.6658333333333),
    "노원구": (127.058388888888, 37.6514611111111),
    "은평구": (126.931241666666, 37.5999694444444),
    "서대문구": (126.938897222222, 37.5763666666666),
    "마포구": (126.910530555555, 37.5607055555555),
    "양천구": (126.868708333333, 37.5142305555555),
    "강서구": (126.851675, 37.5481555555555),
    "구로구": (126.889597222222, 37.49265),
    "금천구": (126.904197222222, 37.4491083333333),
    "영등포구": (126.898341666666, 37.5236111111111),
    "동작구": (126.941575, 37.5096555555555),
    "관악구": (126.953844444444, 37.4753861111111),
    "서초구": (127.034811111111, 37.4807861111111),
    "강남구": (127.049555555555, 37.514575),
    "송파구": (127.107930555555, 37.5117555555555),
    "강동구": (127.125863888888, 37.5273666666666)
}

# 좌표 DataFrame 만들기
df_coordinates = pd.DataFrame.from_dict(coordinates, orient='index', columns=['lon', 'lat'])

# 'MSRSTE_NM' 컬럼에 따라 좌표를 매핑하여 lon, lat 열 추가
df_filled['lon'] = df_filled['MSRSTE_NM'].map(df_coordinates['lon'])
df_filled['lat'] = df_filled['MSRSTE_NM'].map(df_coordinates['lat'])

# NaN 값이 있을 경우 0으로 대체
df_filled['lon'] = df_filled['lon'].fillna(0)
df_filled['lat'] = df_filled['lat'].fillna(0)

# 최종 데이터프레임 출력
column_order = [
    'lat', 'lon', 'year', 'month', 'day', 'hour', 
    'weekday', 'NO2', 'O3', 'CO', 'SO2', 'PM10', 'PM25'
]

df_filled = df_filled[column_order]

columns_round1 = ['CO', 'PM10', 'PM25']
df_filled[columns_round1] = df_filled[columns_round1].round(1)

columns_round3 = ['NO2', 'O3', 'SO2']
df_filled[columns_round3] = df_filled[columns_round3].round(3)

df_filled.columns = df_filled.columns.str.lower() # 컬럼 이름을 소문자로 변경

print(df_filled.head()) 

# 전처리된 데이터프레임을 새로운 CSV 파일로 저장
df_filled.to_csv('seoul_air_quality_ready.csv', index=False)